In [1]:
import pandas as pd
import numpy as np
import cv2
import os, sys
from tqdm import tqdm

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [3]:
def read_img(img_path):
    img = cv2.imread(img_path, cv2.IMREAD_COLOR)
    img = cv2.resize(img, (64,64))
    return img

In [4]:
TRAIN_PATH = 'train_img/'
TEST_PATH = 'test_img/'

In [5]:
train_img, test_img = [],[]
for img_path in tqdm(train['image_id'].values):
    train_img.append(read_img(TRAIN_PATH + img_path + '.png'))

for img_path in tqdm(test['image_id'].values):
    test_img.append(read_img(TEST_PATH + img_path + '.png'))

100%|██████████| 1732/1732 [00:46<00:00, 37.44it/s]


In [6]:
x_train = np.array(train_img, np.float32) / 255.
x_test = np.array(test_img, np.float32) / 255.

In [7]:
label_list = train['label'].tolist()
Y_train = {k:v+1 for v,k in enumerate(set(label_list))}
y_train = [Y_train[k] for k in label_list]   
y_train = np.array(y_train)

In [8]:
from keras import applications
from keras.models import Model
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.metrics import categorical_accuracy
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping
from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint

Using TensorFlow backend.


In [9]:
y_train = to_categorical(y_train)

In [10]:
base_model = applications.VGG16(weights='imagenet', include_top=False, input_shape=(64, 64, 3))

In [11]:
add_model = Sequential()
add_model.add(Flatten(input_shape=base_model.output_shape[1:]))
add_model.add(Dense(256, activation='relu'))
add_model.add(Dense(y_train.shape[1], activation='softmax'))

model = Model(inputs=base_model.input, outputs=add_model(base_model.output))
model.compile(loss='categorical_crossentropy', optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 64, 64, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 64, 64, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 64, 64, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 32, 32, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 32, 32, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 32, 32, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 16, 16, 128)       0         
__________

In [12]:
batch_size = 16 # tune it
epochs = 10 # increase it

train_datagen = ImageDataGenerator(
        rotation_range=30, 
        width_shift_range=0.1,
        height_shift_range=0.1, 
        horizontal_flip=True)
train_datagen.fit(x_train)

In [13]:
history = model.fit_generator(
    train_datagen.flow(x_train, y_train, batch_size=batch_size),
    steps_per_epoch=x_train.shape[0] // batch_size,
    epochs=epochs,
    callbacks=[ModelCheckpoint('VGG16-transferlearning.model', monitor='val_acc', save_best_only=True)]
)

Epoch 1/10
200/200 [==============================] - 383s - loss: 3.1263 - acc: 0.1163   

/usr/local/src/conda3_runtime.v18/4.1.1/lib/python3.5/site-packages/keras/callbacks.py:405: RuntimeWarning: Can save best model only with val_acc available, skipping.
  'skipping.' % (self.monitor), RuntimeWarning)



Epoch 2/10
200/200 [==============================] - 382s - loss: 2.6372 - acc: 0.2366   
Epoch 3/10
200/200 [==============================] - 390s - loss: 2.2479 - acc: 0.3295   
Epoch 4/10
200/200 [==============================] - 403s - loss: 1.9801 - acc: 0.4198   
Epoch 5/10
200/200 [==============================] - 402s - loss: 1.7589 - acc: 0.4708   
Epoch 6/10
200/200 [==============================] - 392s - loss: 1.6019 - acc: 0.5141   
Epoch 7/10
200/200 [==============================] - 387s - loss: 1.4666 - acc: 0.5567   
Epoch 8/10
200/200 [==============================] - 386s - loss: 1.3192 - acc: 0.6102   
Epoch 9/10
200/200 [==============================] - 390s - loss: 1.1960 - acc: 0.6239   
Epoch 10/10
200/200 [==============================] - 385s - loss: 1.1173 - acc: 0.6536   


In [14]:
predictions = model.predict(x_test)

In [15]:
predictions = np.argmax(predictions, axis=1)
rev_y = {v:k for k,v in Y_train.items()}
pred_labels = [rev_y[k] for k in predictions]

In [16]:
vgg = pd.DataFrame({'image_id':test.image_id, 'label':pred_labels})
vgg.to_csv('sub_vgg.csv', index=False)

In [18]:
from IPython.display import HTML
import base64
def create_download_link( df, title = "Download CSV file", filename = "sub_vgg.csv"):  
    csv = df.to_csv(index=False)
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)

create_download_link(vgg)  